This notebook will be used to create biochem open boundary conditions, starting with SJDF, for full domain. 

In [1]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import os
import re

%matplotlib inline

N2chl=1.600

# Load 3D T+S


In [2]:
# define constant values, not yet based on data:
val_NO3=100.
val_NH4=100.
val_Si=100.
val_bSi=100.
val_DIA=100.
val_CRY=100.
val_MYRI=100.
val_MICZ=100.
val_PON=100.
val_DON=100.
val_Oxy = 100.0

In [3]:
TS = nc.Dataset('/data/dlatorne/MEOPAR/nowcast-green/open_boundaries/west/SalishSea_west_TEOS10.nc')

In [4]:
s=''
for key in TS.variables.keys():
    s=s+key+', '
print(s)

deptht, nav_lat, nav_lon, nbidta, nbjdta, nbrdta, time_counter, vosaline, votemper, 


In [5]:
print(TS.dimensions)

OrderedDict([('deptht', <class 'netCDF4._netCDF4.Dimension'>: name = 'deptht', size = 40
), ('yb', <class 'netCDF4._netCDF4.Dimension'>: name = 'yb', size = 1
), ('xbT', <class 'netCDF4._netCDF4.Dimension'>: name = 'xbT', size = 870
), ('time_counter', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time_counter', size = 52
)])


#Save to netcdf

In [6]:
new = nc.Dataset('/ocean/eolson/MEOPAR/NEMO-3.6-inputs/boundary_conditions/bioOBC_full_highvals.nc', 'w', zlib=True)
#Copy dimensions
for dname, the_dim in TS.dimensions.items():
    #print (dname, len(the_dim) if not the_dim.isunlimited() else None)
    new.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)

In [7]:
print(new.dimensions)

OrderedDict([('deptht', <class 'netCDF4._netCDF4.Dimension'>: name = 'deptht', size = 40
), ('yb', <class 'netCDF4._netCDF4.Dimension'>: name = 'yb', size = 1
), ('xbT', <class 'netCDF4._netCDF4.Dimension'>: name = 'xbT', size = 870
), ('time_counter', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time_counter', size = 0
)])


In [8]:
# create dimension variables:
# deptht
deptht=new.createVariable('deptht','float',('deptht',),zlib=True)
deptht.long_name = 'Vertical T Levels'
deptht.units = 'm'
deptht.positive = 'down'
deptht.valid_range = np.array((4., 428.))
deptht[:]=TS.variables['deptht']

#nav_lat
nav_lat = new.createVariable('nav_lat','float',('yb','xbT'),zlib=True)
nav_lat.long_name = TS.variables['nav_lat'].long_name
nav_lat.units = TS.variables['nav_lat'].units
nav_lat[:] = TS.variables['nav_lat']

#nav_lon
nav_lon = new.createVariable('nav_lon','float',('yb','xbT'),zlib=True)
nav_lon.long_name = TS.variables['nav_lon'].long_name
nav_lon.units = TS.variables['nav_lon'].units
nav_lon[:]=TS.variables['nav_lon']

# nbidta
nbidta=new.createVariable('nbidta','int',('yb','xbT'),zlib=True)
nbidta.long_name = TS.variables['nbidta'].long_name
nbidta.units = TS.variables['nbidta'].units
nbidta[:]=TS.variables['nbidta']

# nbjdta
nbjdta=new.createVariable('nbjdta','int',('yb','xbT'),zlib=True)
nbjdta.long_name = TS.variables['nbjdta'].long_name
nbjdta.units = TS.variables['nbjdta'].units
nbjdta[:]=TS.variables['nbjdta']

# nbrdta
nbrdta=new.createVariable('nbrdta','int',('yb','xbT'),zlib=True)
nbrdta.long_name = TS.variables['nbrdta'].long_name
nbrdta.units = TS.variables['nbrdta'].units
nbrdta[:]=TS.variables['nbrdta']

# time_counter
time_counter = new.createVariable('time_counter', 'float', ('time_counter'),zlib=True)
time_counter.long_name = 'Time axis'
time_counter.axis = 'T'
time_counter.units = 'weeks since beginning of year'
time_counter[:]=TS.variables['time_counter']

In [9]:
print(TS.variables['vosaline'])

<class 'netCDF4._netCDF4.Variable'>
float32 vosaline(time_counter, deptht, yb, xbT)
    coordinates: nav_lon nav_lat deptht time_counter
    grid: SalishSea
    units: g/kg
    long_name: Reference Salinity
unlimited dimensions: time_counter
current shape = (52, 40, 1, 870)
filling on, default _FillValue of 9.969209968386869e+36 used



In [10]:
# variables
#NO3, Si, NH4, PHY, PHY2, MYRI, MICZ, POC, DOC, bSi

# NO3
voNO3 = new.createVariable('NO3', 'float32', 
                               ('time_counter','deptht','yb','xbT'),zlib=True)
voNO3.grid = TS.variables['votemper'].grid
voNO3.units = 'muM'
voNO3.long_name = 'Nitrate' 
voNO3[:]=val_NO3


#Si
voSi = new.createVariable('Si', 'float32', 
                               ('time_counter','deptht','yb','xbT'),zlib=True)
voSi.grid = TS.variables['votemper'].grid
voSi.units = 'muM'
voSi.long_name = 'Silica' 
voSi[:]=val_Si


#NH4
voNH4 = new.createVariable('NH4', 'float32', 
                               ('time_counter','deptht','yb','xbT'),zlib=True)
voNH4.grid = TS.variables['votemper'].grid
voNH4.units = 'muM'
voNH4.long_name = 'Ammonia' 
voNH4[:]=val_NH4


#DIA
voDIA = new.createVariable('DIA', 'float32', 
                               ('time_counter','deptht','yb','xbT'),zlib=True)
voDIA.units = 'muM N'
voDIA.long_name = 'Diatoms'
voDIA.grid = TS.variables['votemper'].grid
voDIA[:]=val_DIA

#CRY
voCRY = new.createVariable('CRY', 'float32', 
                               ('time_counter','deptht','yb','xbT'),zlib=True)
voCRY.units = 'muM N'
voCRY.long_name = 'Cryptophytes'
voCRY.grid = TS.variables['votemper'].grid
voCRY[:]=val_CRY

#MYRI
voMYRI = new.createVariable('MYRI', 'float32', 
                               ('time_counter','deptht','yb','xbT'),zlib=True)
voMYRI.units = 'muM N'
voMYRI.long_name = 'M. rubra' 
voMYRI.grid = TS.variables['votemper'].grid
voMYRI[:]=val_MYRI

#MICZ
voMICZ = new.createVariable('MICZ', 'float32', 
                               ('time_counter','deptht','yb','xbT'),zlib=True)
voMICZ.units = 'muM N'
voMICZ.long_name = 'Microzooplankton' 
voMICZ.grid = TS.variables['votemper'].grid
voMICZ[:]=val_MICZ

#PON
voPON = new.createVariable('PON', 'float32', 
                               ('time_counter','deptht','yb','xbT'),zlib=True)
voPON.units = 'muM N'
voPON.long_name = 'Particulate Organic Nitrogen'
voPON.grid = TS.variables['votemper'].grid
voPON[:] = val_PON

#DON
voDON = new.createVariable('DON', 'float32', 
                               ('time_counter','deptht','yb','xbT'),zlib=True)
voDON.units = 'muM N'
voDON.long_name = 'Dissolved Organic Nitrogen'
voDON.grid = TS.variables['votemper'].grid
voDON[:]=val_DON

#bSi
vobSi = new.createVariable('bSi', 'float32', 
                               ('time_counter','deptht','yb','xbT'),zlib=True)
vobSi.units = 'muM N'
vobSi.long_name = 'Biogenic Silica'
vobSi.grid = TS.variables['votemper'].grid
vobSi[:]=val_bSi

#O2
voO2 = new.createVariable('O2', 'float32', 
                               ('time_counter','deptht','yb','xbT'),zlib=True)
#voO2.units = ''
voO2.long_name = 'oxygen'
voO2.grid = TS.variables['votemper'].grid
voO2[:]=val_Oxy


In [11]:
new.close()

In [12]:
TS.close()

# Northern OBCs:

In [2]:
TS = nc.Dataset('/data/dlatorne/MEOPAR/nowcast-green/open_boundaries/north/SalishSea_north_TEOS10.nc')

In [14]:
new = nc.Dataset('/ocean/eolson/MEOPAR/NEMO-3.6-inputs/boundary_conditions/bioOBC_full_north_highvals.nc', 'w', zlib=True)
#Copy dimensions
for dname, the_dim in TS.dimensions.items():
    #print (dname, len(the_dim) if not the_dim.isunlimited() else None)
    new.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)

In [15]:
# create dimension variables:
# deptht
deptht=new.createVariable('deptht','float32',('deptht',))
deptht.long_name = 'Vertical T Levels'
deptht.units = 'm'
deptht.positive = 'down'
deptht.valid_range = np.array((4., 428.))
deptht[:]=TS.variables['deptht']

#nav_lat
nav_lat = new.createVariable('nav_lat','float32',('yb','xbT'))
nav_lat.long_name = TS.variables['nav_lat'].long_name
nav_lat.units = TS.variables['nav_lat'].units
nav_lat[:] = TS.variables['nav_lat']

#nav_lon
nav_lon = new.createVariable('nav_lon','float32',('yb','xbT'))
nav_lon.long_name = TS.variables['nav_lon'].long_name
nav_lon.units = TS.variables['nav_lon'].units
nav_lon[:]=TS.variables['nav_lon']

# nbidta
nbidta=new.createVariable('nbidta','int32',('yb','xbT'))
nbidta.long_name = TS.variables['nbidta'].long_name
nbidta.units = TS.variables['nbidta'].units
nbidta[:]=TS.variables['nbidta']

# nbjdta
nbjdta=new.createVariable('nbjdta','int32',('yb','xbT'))
nbjdta.long_name = TS.variables['nbjdta'].long_name
nbjdta.units = TS.variables['nbjdta'].units
nbjdta[:]=TS.variables['nbjdta']

# nbrdta
nbrdta=new.createVariable('nbrdta','int32',('yb','xbT'))
nbrdta.long_name = TS.variables['nbrdta'].long_name
nbrdta.units = TS.variables['nbrdta'].units
nbrdta[:]=TS.variables['nbrdta']

# time_counter
time_counter = new.createVariable('time_counter', 'float32', ('time_counter'))
time_counter.long_name = 'Time axis'
time_counter.axis = 'T'
time_counter.units = 'weeks since beginning of year'
time_counter[:]=TS.variables['time_counter']

In [16]:
# variables
#NO3, Si, NH4, PHY, PHY2, MYRI, MICZ, POC, DOC, bSi

# NO3
voNO3 = new.createVariable('NO3', 'float32', 
                               ('time_counter','deptht','yb','xbT'))
voNO3.grid = TS.variables['votemper'].grid
voNO3.units = 'muM'
voNO3.long_name = 'Nitrate' 
voNO3[:]=val_NO3

#Si
voSi = new.createVariable('Si', 'float32', 
                               ('time_counter','deptht','yb','xbT'))
voSi.grid = TS.variables['votemper'].grid
voSi.units = 'muM'
voSi.long_name = 'Silica' 
voSi[:]=val_Si

#NH4
voNH4 = new.createVariable('NH4', 'float32', 
                               ('time_counter','deptht','yb','xbT'))
voNH4.grid = TS.variables['votemper'].grid
voNH4.units = 'muM'
voNH4.long_name = 'Ammonia' 
voNH4=val_NH4

#DIA
voDIA = new.createVariable('DIA', 'float32', 
                               ('time_counter','deptht','yb','xbT'))
voDIA.units = 'muM N'
voDIA.long_name = 'Diatoms'
voDIA.grid = TS.variables['votemper'].grid
voDIA[:]=val_DIA

#CRY
voCRY = new.createVariable('CRY', 'float32', 
                               ('time_counter','deptht','yb','xbT'))
voCRY.units = 'muM N'
voCRY.long_name = 'Cryptophytes'
voCRY.grid = TS.variables['votemper'].grid
voCRY[:]=val_CRY

#MYRI
voMYRI = new.createVariable('MYRI', 'float32', 
                               ('time_counter','deptht','yb','xbT'))
voMYRI.units = 'muM N'
voMYRI.long_name = 'M. rubra' 
voMYRI.grid = TS.variables['votemper'].grid
voMYRI[:]=val_MYRI

#MICZ
voMICZ = new.createVariable('MICZ', 'float32', 
                               ('time_counter','deptht','yb','xbT'))
voMICZ.units = 'muM N'
voMICZ.long_name = 'Microzooplankton' 
voMICZ.grid = TS.variables['votemper'].grid
voMICZ[:]=val_MICZ

#PON
voPON = new.createVariable('PON', 'float32', 
                               ('time_counter','deptht','yb','xbT'))
voPON.units = 'muM N'
voPON.long_name = 'Particulate Organic Nitrogen'
voPON.grid = TS.variables['votemper'].grid
voPON[:] = val_PON

#DON
voDON = new.createVariable('DON', 'float32', 
                               ('time_counter','deptht','yb','xbT'))
voDON.units = 'muM N'
voDON.long_name = 'Dissolved Organic Nitrogen'
voDON.grid = TS.variables['votemper'].grid
voDON[:]=val_DON

#bSi
vobSi = new.createVariable('bSi', 'float32', 
                               ('time_counter','deptht','yb','xbT'))
vobSi.units = 'muM N'
vobSi.long_name = 'Biogenic Silica'
vobSi.grid = TS.variables['votemper'].grid
vobSi[:]=val_bSi

#O2
voO2 = new.createVariable('O2', 'float32', 
                               ('time_counter','deptht','yb','xbT'))
#voO2.units = ''
voO2.long_name = 'oxygen'
voO2.grid = TS.variables['votemper'].grid
voO2[:]=val_Oxy


In [17]:
new.close()

In [18]:
TS.close()

In [19]:
f=nc.Dataset('/ocean/eolson/MEOPAR/NEMO-3.6-inputs/boundary_conditions/bioOBC_full_highvals.nc')

In [20]:
print(f.variables)

OrderedDict([('deptht', <class 'netCDF4._netCDF4.Variable'>
float64 deptht(deptht)
    long_name: Vertical T Levels
    units: m
    positive: down
    valid_range: [   4.  428.]
unlimited dimensions: 
current shape = (40,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('nav_lat', <class 'netCDF4._netCDF4.Variable'>
float64 nav_lat(yb, xbT)
    long_name: Latitude
    units: degrees_north
unlimited dimensions: 
current shape = (1, 870)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('nav_lon', <class 'netCDF4._netCDF4.Variable'>
float64 nav_lon(yb, xbT)
    long_name: Longitude
    units: degrees_east
unlimited dimensions: 
current shape = (1, 870)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('nbidta', <class 'netCDF4._netCDF4.Variable'>
int64 nbidta(yb, xbT)
    long_name: i grid position
    units: 1
unlimited dimensions: 
current shape = (1, 870)
filling on, default _FillValue of -9223372036854775806 used
), ('nbjdta', <class 'n

In [21]:
f.close()

In [22]:
f=nc.Dataset('/data/eolson/MEOPAR/SS36runs/run08dec15_2OBC/bio_OBC/bioOBC_full.nc')

In [23]:
print(f.variables['NO3'][1,5,0,10])

100.0


In [24]:
print(f.variables['NH4'][1,5,0,10])

100.0


In [3]:
print(TS.dimensions)

OrderedDict([('xbT', <class 'netCDF4._netCDF4.Dimension'>: name = 'xbT', size = 300
), ('yb', <class 'netCDF4._netCDF4.Dimension'>: name = 'yb', size = 1
), ('deptht', <class 'netCDF4._netCDF4.Dimension'>: name = 'deptht', size = 40
), ('time_counter', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time_counter', size = 2
)])


In [15]:
print(TS.variables)

OrderedDict([('deptht', <class 'netCDF4._netCDF4.Variable'>
float32 deptht(deptht)
    long_name: Vertical T Levels
    units: m
    positive: down
    valid_range: [   4.  428.]
unlimited dimensions: 
current shape = (40,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('time_counter', <class 'netCDF4._netCDF4.Variable'>
float32 time_counter(time_counter)
    long_name: Time axis
    axis: T
    units: half years
unlimited dimensions: time_counter
current shape = (2,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('nav_lat', <class 'netCDF4._netCDF4.Variable'>
float32 nav_lat(yb, xbT)
    long_name: Latitude
    units: degrees_north
unlimited dimensions: 
current shape = (1, 300)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('nav_lon', <class 'netCDF4._netCDF4.Variable'>
float32 nav_lon(yb, xbT)
    long_name: Longitude
    units: degrees_east
unlimited dimensions: 
current shape = (1, 300)
filling on, default _FillValue of 9.96920

In [8]:
print(np.max(TS.variables['nbjdta'][:,0:20]))

896
